In [ ]:
import os
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
!unlink "/content/coopmotion"
!ln -s "/gdrive/My Drive/coopmotion/" "/content/coopmotion"
!rm -r "/content/coopmotion/dataset (1)" "/content/coopmotion/dataset" "/content/coopmotion/models"

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')
sys.path.append('coopmotion/src')
from matplotlib import pyplot as plt
from task.fancy import animate_with_targets, plot_trace
import numpy as np
import torch
import time

# Learning

In [7]:
from task.dataset import generate_non_sequential_dataset, generate_randomstart_dataset, central_dataset
from task.square import static_evenly_spread_task, SegmentRun, proportional, sense_in_range
controller = proportional
N = 4
number_of_samples = 1
eps = 0.001
task = static_evenly_spread_task(N)
run = SegmentRun(task, controller=controller, sensor=sense_in_range(6), dt=0.1)
train_dataset = central_dataset(generate_non_sequential_dataset(run=run, number=number_of_samples, epsilon=eps))
test_dataset = central_dataset(generate_non_sequential_dataset(run=run, number=number_of_samples, epsilon=eps))

(120, 4, 3) (120, 4, 2)
[9.98907644e-01 4.98868998e-01 4.76786407e-04 5.01288281e-01] [5.01837495e-01 9.97781317e-01 4.97729626e-01 4.78981727e-04] [-1.03444479  1.09936899  1.77779186 -2.7856311 ]
[[ 9.98907644e-01  5.01837495e-01 -1.03444479e+00]
 [ 4.98868998e-01  9.97781317e-01  1.09936899e+00]
 [ 4.76786407e-04  4.97729626e-01  1.77779186e+00]
 [ 5.01288281e-01  4.78981727e-04 -2.78563110e+00]]
[[ 1.06883471e-03  1.54163788e-13]
 [ 1.24516270e-03 -4.88301590e-14]
 [ 1.15994854e-03 -4.04592746e-14]
 [ 6.87220931e-04 -3.01351838e-13]]
(135, 4, 3) (135, 4, 2)
[0.96281074 0.65939228 0.56285969 0.68275479] [0.6303096  0.77404211 0.4718244  0.2256197 ] [-1.20920234  2.46950797 -3.10903827 -2.24284195]
[[ 0.96281074  0.6303096  -1.20920234]
 [ 0.65939228  0.77404211  2.46950797]
 [ 0.56285969  0.4718244  -3.10903827]
 [ 0.68275479  0.2256197  -2.24284195]]
[[ 0.06775624 -0.26261433]
 [ 0.1382596  -0.89336218]
 [ 0.28178223 -0.25940312]
 [ 0.14517538 -0.02755422]]


## Centralized Network

In [ ]:
from network import CentralizedNet, train_net
net = CentralizedNet(4)
training_loss, testing_loss = [], []
train_net(epochs=500, net=net, train_dataset=train_dataset, test_dataset=test_dataset, batch_size=100, 
          training_loss=training_loss, testing_loss=testing_loss);

In [ ]:
plt.title('Loss')
plt.semilogy(training_loss, label='training')
plt.semilogy(testing_loss, label='testing')
plt.xlabel('epoch')
plt.legend()
plt.savefig('centralized_train_loss')

In [ ]:
from task import Run
from task.square import dynamic, sense_in_range
net_controller = net.controller()
net_run = Run(task, dynamic=dynamic(dt=0.1), sensor=run.sensor, dt=0.1, controller=net_controller)
trace = net_run(T=10, epsilon=0.01)
plot_trace(trace, task)

In [ ]:
animate_with_targets(trace, sensor=net_run.sensor, save_to='centralized_hungarian.mp4')

### How well are we controlling the agents?

In [ ]:
from task.dataset import generate_sequential_dataset
exact_traces = generate_sequential_dataset(run, number=100, duration=10, epsilon=0.01)
net_traces = generate_sequential_dataset(net_run, number=100, duration=10, epsilon=0.01)
from task import prepare
exact_traces = [prepare(t, padding=True, steps=50) for t in exact_traces]
net_traces = [prepare(t, padding=True, steps=50) for t in net_traces]
time = exact_traces[0].time
exact_error = np.array([t.error for t in exact_traces])
net_error = np.array([t.error for t in net_traces])

In [ ]:
def plot_error(time, error, **kwargs):
    m = np.mean(error, axis=0)
    q95 = np.quantile(error, 0.95, axis=0)
    q5 = np.quantile(error, 0.05, axis=0)
    label = kwargs.pop('label', '')
    plt.plot(time, m, '-', label=label, **kwargs)
    plt.fill_between(time, q5, q95, alpha=0.1, **kwargs)
plot_error(time, exact_error, color='black', label='exact')
plot_error(time, net_error, color='blue', label='net')
plt.legend()
plt.savefig('centralized_hungarian_error_comp')
plt.title('Error');

## Distributed Network

In [ ]:
from task.square import SegmentRun, smart_static_evenly_spread_task, dynamic_evenly_spread_task, sense_in_range
from task.dataset import distributed_dataset

In [ ]:
N = 4
number_of_samples = 10
EPS = 0.001
task = dynamic_evenly_spread_task(N)
#generate_non_sequential_dataset(run=run, number=number_of_samples, epsilon=0.00001)
run = SegmentRun(task, controller=controller, sensor=sense_in_range(0.7), dt=0.1)
d_train_dataset = distributed_dataset(generate_non_sequential_dataset(run=run, number=number_of_samples, epsilon=EPS))
d_test_dataset = distributed_dataset(generate_non_sequential_dataset(run=run, number=number_of_samples, epsilon=EPS))
#d_train_dataset = distributed_dataset(generate_randomstart_dataset(run=run, number=number_of_samples))
#d_test_dataset = distributed_dataset(generate_randomstart_dataset(run=run, number=number_of_samples))

In [ ]:
from network import DistributedNet
d_net = DistributedNet(run.sensor.get_input_size(N))
d_training_loss, d_testing_loss = [], []
train_net(epochs=500, net=d_net, train_dataset=d_train_dataset, test_dataset=d_test_dataset, batch_size=100, 
          training_loss=d_training_loss, testing_loss=d_testing_loss);

In [ ]:
d_net_controller = d_net.controller()
d_net_run = Run(task, dynamic=dynamic(dt=0.1), sensor=run.sensor, dt=0.1, controller=d_net_controller)
trace = d_net_run(T=25, epsilon=0.01)
plot_trace(trace, task)

In [ ]:
animate_with_targets(trace, sensor=net_run.sensor)

In [ ]:
d_net_traces = generate_sequential_dataset(d_net_run, number=100, duration=10, epsilon=0.01)
d_net_traces = [prepare(t, padding=True, steps=50) for t in d_net_traces]
d_net_error = np.array([t.error for t in d_net_traces])

In [ ]:
plot_error(time, exact_error, color='black', label='exact')
plot_error(time, net_error, color='blue', label='net')
plot_error(time, d_net_error, color='green', label='d_net')
plt.legend()
plt.title('Error');

## Distributed Communicating Network

In [ ]:
from task.dataset import sequence_dataset
from task.square import smart_static_evenly_spread_task, sense_in_range, SegmentRun, proportional, static_zipf_task

In [ ]:
number_of_samples = 10
run = SegmentRun(smart_static_evenly_spread_task(4), controller=controller, sensor=sense_in_range(0.8), dt=0.1)
train_traces = generate_sequential_dataset(run=run, number=number_of_samples)
test_traces = generate_sequential_dataset(run=run, number=number_of_samples)

In [ ]:
c_train_dataset = sequence_dataset(train_traces, steps=2)
c_test_dataset = sequence_dataset(test_traces, steps=2)
c_train_dataset[50]

In [ ]:
from com_network import ComNet

In [ ]:
c_net = ComNet(N=N, broadcast=2)
c_training_loss, c_testing_loss = [], []

In [ ]:
train_net(epochs=100, net=c_net, train_dataset=c_train_dataset, test_dataset=c_test_dataset, batch_size=100, 
          training_loss=c_training_loss, testing_loss=c_testing_loss);

In [ ]:
plt.title('Loss')
plt.semilogy(c_training_loss, label='training')
plt.semilogy(c_testing_loss, label='testing')
plt.xlabel('epoch')
plt.legend()

In [ ]:
c_net_controller = c_net.controller()
c_net_run = Run(task, dynamic=dynamic(dt=0.1), sensor=run.sensor, dt=0.1, controller=c_net_controller)

In [ ]:
trace = c_net_run(T=5)
plot_trace(trace,task)

In [ ]:
animate_with_targets(trace, sensor=net_run.sensor)

In [ ]:
plt.plot(trace.control[:,:,0], trace.communication[:,0,:], '.')

In [ ]:
c_net_traces = generate_sequential_dataset(c_net_run, number=100, duration=5, epsilon=0.01)

In [ ]:
c_net_traces = [prepare(t, padding=True, steps=50) for t in c_net_traces]

In [ ]:
c_net_error = np.array([t.error for t in c_net_traces])

In [ ]:
plot_error(time, exact_error, color='black', label='exact')
plot_error(time, net_error, color='blue', label='net')
plot_error(time, d_net_error, color='green', label='d_net')
plot_error(time, c_net_error, color='orange', label='c_net')
plt.legend()
plt.title('Error');